In [2]:
from activations import LeakyRelu, Tanh, Sigmoid

l = [LeakyRelu()]

In [12]:
for i in l:
    print('activations' in   """<class 'activations.LeakyRelu'>""" )

True


In [ ]:

from VAE import Encoder, Decoder, Discriminator, VAEGANTrainer
from tensorflow.keras.datasets import mnist
import numpy as np
# Инициализация
# Инициализация
# Подготовка данных
def preprocess_data(x):
    x = x.astype('float32') / 255.0
    return x.reshape(x.shape[0], -1)

# Загрузка данных MNIST
(x_train, _), (x_test, _) = mnist.load_data()
x_train = preprocess_data(x_train)

# Инициализация моделей
encoder = Encoder()
decoder = Decoder()

# Обучение
trainer = VAEGANTrainer(encoder, decoder, learning_rate=1e-4)
trainer.train(x_train, batch_size=128, epochs=50)


Epoch 1/50 - VAE Loss: 32637916.8000, GAN Loss: 0.0000, KL Loss: 137.9643
Epoch 2/50 - VAE Loss: 32607909.5054, GAN Loss: 0.0000, KL Loss: 2266.1202
Epoch 3/50 - VAE Loss: 32612988.8535, GAN Loss: 0.0000, KL Loss: 7345.4683
Epoch 4/50 - VAE Loss: 32620985.3628, GAN Loss: 0.0000, KL Loss: 15341.9776
Epoch 5/50 - VAE Loss: 32632039.0455, GAN Loss: 0.0000, KL Loss: 26395.6603
Epoch 6/50 - VAE Loss: 32645893.5808, GAN Loss: 0.0000, KL Loss: 40250.1956
Epoch 7/50 - VAE Loss: 32662507.3038, GAN Loss: 0.0000, KL Loss: 56863.9186
Epoch 8/50 - VAE Loss: 32681698.8035, GAN Loss: 0.0000, KL Loss: 76055.4183
Epoch 9/50 - VAE Loss: 32703346.2760, GAN Loss: 0.0000, KL Loss: 97702.8908
Epoch 10/50 - VAE Loss: 32727375.5095, GAN Loss: 0.0000, KL Loss: 121732.1243
Epoch 11/50 - VAE Loss: 32753734.5825, GAN Loss: 0.0000, KL Loss: 148091.1973
Epoch 12/50 - VAE Loss: 32782385.2706, GAN Loss: 0.0000, KL Loss: 176741.8854
Epoch 13/50 - VAE Loss: 32813298.4080, GAN Loss: 0.0000, KL Loss: 207655.0228
Epoch 14

In [ ]:
def plot_latent_space(model, scale=1.0, n=25, digit_size=28, figsize=15):
    # display a n*n 2D manifold of digits
    figure = np.zeros((digit_size * n, digit_size * n))

    # construct a grid 
    grid_x = np.linspace(-scale, scale, n)
    grid_y = np.linspace(-scale, scale, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = torch.tensor([[xi, yi]], dtype=torch.float).to(device)
            x_decoded = model.decode(z_sample)
            digit = x_decoded[0].detach().cpu().reshape(digit_size, digit_size)
            figure[i * digit_size : (i + 1) * digit_size, j * digit_size : (j + 1) * digit_size,] = digit

    plt.figure(figsize=(figsize, figsize))
    plt.title('VAE Latent Space Visualization')
    start_range = digit_size // 2
    end_range = n * digit_size + start_range
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("mean, z [0]")
    plt.ylabel("var, z [1]")
    plt.imshow(figure, cmap="Greys_r")
    plt.show()


plot_latent_space(model)

dout shape: (128, 784), self.a_l shape: (128, 1024)
dout shape: (128, 1024), self.a_l shape: (128, 1024)
dout shape: (128, 1024), self.a_l shape: (128, 1024)
dout shape: (128, 1024), self.a_l shape: (128, 20)


array([[-0.24108571, -1.40441918,  1.82737838, ..., -2.78602793,
        -2.71201002, -3.79868253],
       [ 1.91138255,  2.94798126, -3.89004005, ..., -2.00948048,
        -0.07226309,  0.03223549],
       [ 0.43435469,  0.75003593,  0.22564978, ...,  1.34240311,
        -2.86358409,  0.20198908],
       ...,
       [ 3.65323809, -2.95481143,  0.89444766, ...,  0.03137546,
        -1.8165209 ,  0.94955529],
       [ 0.89875342, -0.08134535,  3.45471377, ..., -0.54956157,
        -9.22691528,  3.7789225 ],
       [ 2.94259757, -2.44156705,  0.5699836 , ..., -0.17115144,
         2.22437951, -3.96161648]])

In [5]:
ps.shape

(128, 784)